percent_true
has_ergot indicates if there is at least one true
sum_severity
previous_one_has_ergot indicates if there is at least one true last year
previous_one_severity
previous_two_has_ergot indicates if there is at least one true two years ago
previous_two_severity

we also need a car_uid column to link the prov - district in this table to the ag region boundary table

the final ergot table has

In [2]:
import sqlalchemy as sq
import geopandas as gpd  # type: ignore
import pandas as pd  # type: ignore
from dotenv import load_dotenv
import os, sys

sys.path.append("../")
from Shared.DataService import DataService  # type: ignore

e:\Programming\CGC_Grain_Outcome_Predictions\env\lib\site-packages\geopandas\_compat.py:124: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
C:\Users\danew\AppData\Local\Temp\ipykernel_23096\3364527155.py:2: DeprecationWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas still uses PyGEOS by default. However, starting with version 0.14, the default will switch to Shapely. To force to use Shapely 2.0 now, you can either uninstall PyGEOS or set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In the next release, GeoPandas will switch to using Shapely by default, even if PyGEOS is installed. If you only have PyGEOS installed to get speed-ups, this switch should be s

In [3]:
load_dotenv()
PG_DB = os.getenv("POSTGRES_DB")
PG_ADDR = os.getenv("POSTGRES_ADDR")
PG_PORT = os.getenv("POSTGRES_PORT")
PG_USER = os.getenv("POSTGRES_USER")
PG_PW = os.getenv("POSTGRES_PW")

In [4]:
db = DataService(PG_DB, PG_ADDR, PG_PORT, PG_USER, PG_PW)
conn = db.connect()

In [5]:
query = "SELECT * FROM public.ergot_sample"
ergotSamples = pd.read_sql_query(query, conn)  # type: ignore
#components = pd.DataFrame(query, columns=["soil_id", "poly_id"])  # type: ignore

In [6]:
ergotSamples

,sample_id,year,province,crop_district,incidence,severity
0,1,1995,AB,1.0,False,NaN
1,2,1995,AB,1.0,False,NaN
2,3,1995,AB,1.0,False,NaN
3,4,1995,AB,1.0,False,NaN
4,5,1995,AB,1.0,False,NaN
...,...,...,...,...,...,...
158694,158695,2022,SK,81.0,False,NaN
158695,158696,2022,SK,81.0,False,NaN
158696,158697,2022,SK,81.0,False,NaN
158697,158698,2022,AB,7.0,False,NaN


In [ ]:
ALTER TABLE public.census_ag_regions ALTER COLUMN car_uid TYPE integer USING car_uid::integer;

In [15]:
# calculate the car_uid
car_uids = []
for index, row in ergotSamples.iterrows():
    uid = row['crop_district']

    if row['province'] == 'MB':
        uid += 4600
    if row['province'] == 'SK':
        uid = uid - 1
        uid += 4700
    if row['province'] == 'AB':
        uid = uid*10
        uid += 4800

    
    car_uids.append(uid)

In [16]:
ergotSamples['car_uid'] = car_uids

In [31]:
ergotSamples

,sample_id,year,province,crop_district,incidence,severity,car_uid
0,1,1995,AB,1.0,False,NaN,4810.0
1,2,1995,AB,1.0,False,NaN,4810.0
2,3,1995,AB,1.0,False,NaN,4810.0
3,4,1995,AB,1.0,False,NaN,4810.0
4,5,1995,AB,1.0,False,NaN,4810.0
...,...,...,...,...,...,...,...
158694,158695,2022,SK,81.0,False,NaN,4780.0
158695,158696,2022,SK,81.0,False,NaN,4780.0
158696,158697,2022,SK,81.0,False,NaN,4780.0
158697,158698,2022,AB,7.0,False,NaN,4870.0


In [ ]:
# year                  ######
# province              ######
# crop_district         ######
# car_uid               ######
# percent_true
# has_ergot indicates if there is at least one true
# sum_severity
# previous_one_has_ergot indicates if there is at least one true last year
# previous_one_severity
# previous_two_has_ergot indicates if there is at least one true two years ago
# previous_two_severity
# previous_three_severity
# bordering agRegion has Ergot